<p style="text-align: right; font-size:0.8em;"> Thomas Bury<br><a href=mailto:thomas.bury@mcgill.ca> thomas.bury@mcgill.ca </a> </p>
<h1> Notebook 4: An introduction to Plotly Dash </h1>
<img src='images/dash_logo.png' width='100'>
Learning objectives of this notebook:

1. Familiarise yourself with the layout of a Dash application
2. Edit a template to create your own application

<br>

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

<br><h2> Pros and cons of using Dash over Plotly </h2>

Pros:
- Huge flexibility with layout akin to making a website
- Connect multiple visualisations via callback functions

Cons:
- Code sligtly more challenging - no more one-liners
- Cannot share as an html file - requires hosting

<br><h2> Creating a minimal Dash app for the Covid data </h2>
<img src='images/covid19.jpg' width='60'>
It would take a separate workshop to teach Dash. The best way to learn it is probably to work through examples and be inspired by the <a href=https://dash-gallery.plotly.host/Portal/>Dash app gallery </a>, many of which have source code available.


Here, we go through the steps required to create a Dash app. We use the Covid data as an example.


<br><h4>1. Import and clean the data </h4>

In [2]:
# Import data
df_covid = pd.read_csv('datasets/covid_19_data.csv')
# Put date of observation into 'datetime' form
df_covid['ObservationDate'] = pd.to_datetime(df_covid['ObservationDate'])
# Fill nan values in Province cells with string 'NA'
df_covid['Province/State'] = df_covid['Province/State'].fillna('NA')

In [3]:
df_covid.head()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,2020-01-22,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,2020-01-22,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,2020-01-22,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,2020-01-22,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,2020-01-22,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0


<br><br><h4>2. Write a function for each plot  </h4>
- The code for your Dash app will be easier to write and more readable if you create functions for each plot you want to include, prior to coding the app.
- The input should be anything you want to interactively vary in the app e.g. the country for Covid cases.
- The output should be the Plotly figure object

In [4]:
def make_covid_tseries(country):
    ''' Create a Plotly figure for the confirmed cases of a country over time'''
    
    # Get country-specific data
    df_plot = df_covid[df_covid['Country/Region']==country]
    
    # Create line plot usingn Plotly express
    fig = px.line(df_plot,
                 x='ObservationDate',
                 y='Confirmed',
                 color='Province/State'
          ) 
    
    # Return the figure object
    return fig

- Make sure to **test** the functions prior to creating the app (easier to debug this way)

In [5]:
fig = make_covid_tseries('Mainland China')
fig.write_html('temp.html')

In [6]:
# Get country names for dropdown menu (sort alphabetically)
country_names = df_covid['Country/Region'].sort_values().unique()
country_names;

<br><br><h4> 3. Define the app </h4>
- Not much to do here. This sets up the app and its CSS style file. If you know CSS you can create your own template. We will just use a premade stylesheet.

In [7]:
app = JupyterDash(__name__,
#                   external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'],
                 )
# Outside of Jupyter (e.g in a Python script), this would be
# app = dash.Dash(__name__)

<br><br><h4>4. Set the app layout </h4>
The app layout lets us
- Set position and dimensions of each app component (e.g plots, dropdown menus, text)
- Assign an ID to each element of the app (important for callback functions, later)

<a href=https://dash.plotly.com/layout> Plotly documentation for Dash layout </a>

In [8]:
app.layout = html.Div([
    
    # Make a title
    html.H1('Covid visualisation'),
    
    # Add a dropdown box
    html.Div(
        [
        # Title of dropdown menu
        html.Label('Country'),
        # Format of dropdown menu
        dcc.Dropdown(id='dropdown_country',
                     options=[{'label': c, 'value': c} for c in country_names],
                     value='Canada', # initial value
                     optionHeight=20,
                     searchable=False,
                     clearable=False),
        ]),
                
    # Add the plot
    html.Div(
        [dcc.Graph(id='covid_graph',
                   figure = make_covid_tseries('Canada')
                  )
        ]
    ),
])

<br><h4> 5. Define callback functions </h4>
Callback functions make the app interactive. They say what to do when a user changes something withing the app, such as an entry in a dropdown menu.

The input defines what the user interacts with to activate this function. It has the form
<br>
<code>Input('ID of component interacted with','value')</code>

The output defines what changes when the user interacts with this component. It has the form
<br>
<code>Output('ID of component that changes','figure')</code>

<a href=https://dash.plotly.com/basic-callbacks> Plotly documentation on basic callbacks </a>

In [9]:
@app.callback(
    Output('covid_graph', 'figure'),
    [Input("dropdown_country", 'value'),
    Input("dropdown_country", 'value')]
)

def update_figure(country,trange):
    fig = make_covid_tseries(country)
    return fig


<br><h4>6. Run the app on your local server</h4>
- mode: 'external'/'internal' - provides link to view app in browser/displays in Jupyter notebook
- debug: True/False - display a button on the app to view error logs
- port: Which local port (on your computer) to run the app on


In [10]:
# Run app 
app.run_server(mode='inline',
               debug=True,
               port=8050)

<br><h2> Choice of exercises (increasing levels of difficulty) </h2>

1. Modify the above Dash app to plot Covid deaths as opposed to cases for each country.
2. Modify the app to display only a subset of countries in the full list.
3. Modify the graph to display a scatter plot total confirmed cases vs total deaths.
4. Create a new dash app below using this template but for a different dataset.